<a href="https://colab.research.google.com/github/yangsoyoung10011001/machinelearning/blob/main/searchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import math
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# -------------------cran.all.1400-------------------
# read cran.all.1400 file
f=open('/content/drive/Shareddrives/머신러닝팀플/cran/cran.all.1400.txt', 'r')
cranAll=f.read()
f.close()

# split by .I
cranAll=cranAll.split('.I')
cranAll.pop(0)

# store each .W in list
cranAllList=[]
for i in range(len(cranAll)):
    cranAllList.append(cranAll[i].split('.W')[1])
# ---------------------------------------------------------

# -------------------cran.qry-------------------
# read cran.qry file
f=open('/content/drive/Shareddrives/머신러닝팀플/cran/cran.qry.txt', 'r')
cranQry=f.read()
f.close()

# split by .I
cranQry=cranQry.split('.I')
cranQry.pop(0)


# store each .W in list
cranQryList=[]
for i in range(len(cranQry)):
    cranQryList.append(cranQry[i].split('.W')[1])
# ---------------------------------------------------------

#--------------------------cranqrel-----------------------------------
f=open('/content/drive/Shareddrives/머신러닝팀플/cran/cranqrel.txt', 'r')
cranqrel=f.read()
f.close()

# split by -1
cranqrel=cranqrel.split('-1')

cranqrelList=[]
for i in range(len(cranqrel)):
    cranqrelList.append(cranqrel[i].split('\n'))
for i in range(1, len(cranqrel)):
  cranqrelList[i].pop(0)

cranqrelListList=[]
for i in range(len(cranqrel)-1):
  tempList=[]
  for j in range(len(cranqrelList[i])):
    tempList.append(cranqrelList[i][j].split(' ')[1])
  tempList.pop(len(tempList)-1)
  cranqrelListList.append(tempList)
#--------------------------------------------------------------------------

FileNotFoundError: ignored

In [ ]:
# list sorting function
def wordSorting(wordList):
  wordList.sort()

  # return sorted list
  return wordList

# function make sorted list of words in query
def makeQueryWordList(query):
    cVec = CountVectorizer(stop_words='english')
    tf = cVec.fit_transform([query])
    queryWordList = list(cVec.vocabulary_)

    # return sorted list of words in query
    return wordSorting(queryWordList)

def makeDocIndexList(tf, totalWordDict, queryWordList, indexOfQuery, docLen):
  docIndexListList=[]
  # query에 있는 단어 개수만큼 for
  for i in queryWordList[indexOfQuery]:
    wordIndex=totalWordDict.get(i)

    docIndexList=[]
        # doc 개수만큼 for
    for j in range(docLen):
            # if(tf[j, wordIndex]==None):
            #     continue
      if(wordIndex!=None):
        if(tf[j, wordIndex]!=0):
          docIndexList.append(j)
    docIndexListList.append(docIndexList)

    # return list of document index for each word in query
  return docIndexListList

# list to store sorted words in each query
q_wordListList=[]
for j in range(len(cranQryList)):
  q_wordListList.append(makeQueryWordList(cranQryList[j]))

# make list for CountVectorizer
cranAllListList=[]
for i in range(len(cranAllList)):
    cranAllListList.append(cranAllList[i])
# compute tf
cVec=CountVectorizer(stop_words='english')
tf=cVec.fit_transform(cranAllListList)
# make list of total words in documents
totalWordList=list(cVec.vocabulary_.keys())
totalWordList=wordSorting(totalWordList)

#input('Enter query index: ')
queryIndex=int(input('Enter query index: '))-1
docIndexListList=makeDocIndexList(tf, cVec.vocabulary_, q_wordListList, queryIndex, len(cranAllListList))

# make inverted index table
df=pd.DataFrame(q_wordListList[queryIndex], columns=['Term'])
df['Document']=docIndexListList
# print(df)

# list to store document index without duplication
totalDocIndexList=[]
for i in range(len(df)):
    for j in range(len(df[i:i+1]['Document'].values[0])):
        totalDocIndexList.append(df[i:i+1]['Document'].values[0][j])


#____________________________remove duplication ______________________________________________

a=list(totalDocIndexList)
a1=list(totalDocIndexList)

a2=[]

for e in range(0,len(a1)):
  k=a1[e]
  if k in a:
    a2.append(k)
    while k in a:
      a.remove(k)  


In [ ]:

###---------------------------------------------

totalDocIndexList=a2

numDoc=len(totalDocIndexList)

89번 sorted(a2) 고치면 값바뀜
query 0번째에서 sorted하면 precision, recall 0나옴
sorted(a2)하지말기


In [ ]:
#-----------------------------------------------------------------
# list to store query and document
listToCVec=[]
# listToCVec.append(cranQryList[queryIndex])
for i in range(numDoc):
    listToCVec.append(cranAllList[totalDocIndexList[i]])
cVec = CountVectorizer(stop_words='english', max_df=0.5)
sf = cVec.fit_transform(listToCVec)
transformer = TfidfTransformer(smooth_idf =False)
transformed_weights= transformer.fit_transform (sf)

query_sf = cVec.transform([cranQryList[queryIndex]])
query_weight = transformer.transform(query_sf)

docW = transformed_weights.toarray()
queryW =query_weight.toarray()

sim=docW*queryW

sim_sum = pd.DataFrame(columns=['Doc','Similarity'])

for j in range(0,numDoc):
  sim_sum=sim_sum.append({'Doc':j+1,'Similarity':sim[j].sum()},ignore_index=True)

order=sim_sum.sort_values('Similarity',ascending=False)
order_list=list(order['Doc'])


In [ ]:
#test
order_list_str=[]
for k in order_list:
  order_list_str.append(str(int(k)))

sameList2=[]
for i in order_list_str:
  if i in cranqrelListList[0]:
    sameList2.append(i)        
  else:
    continue 


여기서 k list받음


In [ ]:
# order=sim_sum.sort_values('Similarity',ascending=False)
# order_list=list(order['Doc'])
# print(order_list)

#----------sort_doc_Ranked by K-----------
def sort_docRank(k_list,order_list2):
  c=[]
  for i in k_list:
    b=order_list2[0:i]
    c.append(b)
  return c
k=[3,5]
rank_doc_query=sort_docRank(k,order_list_str)


In [ ]:
#------------find query num data----------
def extractList_relByquery(queryIndex,k_list):
  rel_list_want=[]
  rel_list_want.append(cranqrelListList[int(queryIndex)])
  return rel_list_want*len(k_list)
  
rel_want_byQuery=extractList_relByquery(queryIndex,k)
print(rel_want_byQuery)

In [ ]:
#------------find false data---------------
#print(doc_list)
## type inverter------
def typeinverter(list_int):
  inverter=[]
  for y in list_int:
    inverter.append(str(y))
  return(inverter)


def falsedata(checkdata):
  doc_list=[]
  for j in range(0,len(checkdata)):
    doc_list.append(typeinverter(list(np.arange(1,1401))))
    for a in checkdata[j]:#check data = cranqrelListList or predictedListList
      doc_list[j].remove(a) 
  # print(doc_list)
  return(doc_list)

# print(r_f)
# print(p_f)


In [ ]:
# #query=2, TT,TF,FT,FF
# def samedata(rel,pred):#rel 데이터 1400개중에 사용할 쿼리번호 ???
#     same_data=[]
#     for i in range(0,len(pred)):
#       same_data.append(list(set(rel[i].intersection(pred[i]))))
#     print(same_data)

def sequence(seq1,seq2,k_list):
  sameList=[]
  # diffList_list1=[]
  # diffList_list2=[]
#seq1과 seq2의 개수는 같을거야 근데 이중리스트일거야
# 근데 바깥[]은 같고==원하는 쿼리개수(len(query_list)), 
#[[]]안은 다를수도있어(쿼리에 해당하는 k개의 doc번호 거나,rel파일의 관련있는 doc번호야)
  for t in range(0,len(k_list)):
    sameList2=[]
    # diffList_list1=[]
    # diffList_list2=[]
    for i in seq1[t]:
      # sameList2=[]
      if i in seq2[t]:
        sameList2.append(i)        
      else:
        continue 
    sameList.append(sameList2)
  # for j in sameList:
  #  print(j)
    # diffList_list1=(list(filter(lambda x:x not in sameList,seq1)))
    # diffList_list2=(list(filter(lambda x:x not in sameList,seq2)))
  
  # print(k)
  return sameList#,diffList_list1,diffList_list2

# print(sequence(r_f,p_f))
# print(sequence(r_f,p_f))
# p(rint(sequence(r_f,p_f)[2])
rf=falsedata(rel_want_byQuery)
# print(rf)
pf=falsedata(rank_doc_query)

rt=rel_want_byQuery
pt=rank_doc_query


rtpt=sequence(rt,pt,k)
rtpf=sequence(rt,pf,k)
rfpt=sequence(rf,pt,k)
rfpf=sequence(rf,pf,k)
# print("tp : ",len(rtpt[0]))
# print("fn : ",len(rtpf[0]))
# print("fp : ",len(rfpt[0]))
# print("tn : ",len(rfpf[0]))


def precision_recall (tp,fn,fp,tn,k_list):
  precision=[]
  recall=[]

  for i in range(0,len(k_list)):
    zerodivision=len(tp[i])+len(fp[i])
    if zerodivision > 0:
      recall.append(len(tp[i])/(len(tp[i])+len(fn[i])))
      precision.append(len(tp[i])/(len(tp[i])+len(fp[i])))
  else:
    print("didn't have true positive data and false positive data")
  return precision,recall


prec=precision_recall(rtpt,rtpf,rfpt,rfpf,k)[0]
rec=precision_recall(rtpt,rtpf,rfpt,rfpf,k)[1]
print('precision:',prec)
print('recall:', rec)

list_df = pd.DataFrame(list(zip(k, prec,rec)),columns=['k', 'precision','recall'])
print(list_df)



In [ ]:
print(list_df.sort_values(by='precision', ascending=False))
print(list_df.sort_values(by='recall', ascending=False))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')